# Training on multiple topologies
Eamon Whalen

In [1]:
import sys
import glob
import numpy as np
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
trainSets ,valSets ,testSets = {}, {}, {}

doeFiles = np.sort(glob.glob("/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/*1000.csv"))
for doeFile in doeFiles:
    designName = doeFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)
    allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
    trainData, valData, testData = partitionGraphList(allGraphs)
    trainSets[designName] = trainData
    valSets[designName] = valData
    testSets[designName] = testData

loading design_5
loading design_6
loading design_7
loading design_8
loading design_9


## 2. Train on each group seperately

In [3]:
resultsList = []
saveDir = './results/topoTest02/'
epochs = 100

In [4]:
for trainName, trainSet in trainSets.items():
    print('training on '+trainName)

    # train
    gcn = FeaStNet()
    history = gcn.trainModel(trainSet, valSets[trainName], epochs=epochs, saveDir=saveDir+trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['test group']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
        
pd.DataFrame(resultsList)

training on design_5
epoch: 0   trainLoss: 1.0237e+00   valLoss:1.0426e+00  time: 1.63e+00
epoch: 1   trainLoss: 7.3107e-01   valLoss:1.0626e+00  time: 1.35e+00
epoch: 2   trainLoss: 5.8352e-01   valLoss:1.2783e+00  time: 1.31e+00
epoch: 3   trainLoss: 4.2983e-01   valLoss:1.8868e+00  time: 1.44e+00
epoch: 4   trainLoss: 2.8272e-01   valLoss:2.2318e+00  time: 1.34e+00
epoch: 5   trainLoss: 2.1404e-01   valLoss:2.9167e+00  time: 1.32e+00
epoch: 6   trainLoss: 1.5903e-01   valLoss:1.5332e+00  time: 1.33e+00
epoch: 7   trainLoss: 1.2663e-01   valLoss:5.2669e-01  time: 1.33e+00
epoch: 8   trainLoss: 1.0605e-01   valLoss:5.1805e-01  time: 1.33e+00
epoch: 9   trainLoss: 9.1138e-02   valLoss:5.3773e-01  time: 1.32e+00
epoch: 10   trainLoss: 1.1481e-01   valLoss:4.7422e-01  time: 1.33e+00
epoch: 11   trainLoss: 7.7063e-02   valLoss:5.1312e-01  time: 1.35e+00
epoch: 12   trainLoss: 8.0997e-02   valLoss:4.5806e-01  time: 1.33e+00
epoch: 13   trainLoss: 5.7019e-02   valLoss:4.9806e-01  time: 1.33

alt.Chart(...)

testing on design_5

training on design_6
epoch: 0   trainLoss: 9.6603e-01   valLoss:1.1957e+00  time: 1.40e+00
epoch: 1   trainLoss: 7.4331e-01   valLoss:1.2062e+00  time: 1.43e+00
epoch: 2   trainLoss: 6.0668e-01   valLoss:1.2946e+00  time: 1.40e+00
epoch: 3   trainLoss: 4.9623e-01   valLoss:2.0733e+00  time: 1.42e+00
epoch: 4   trainLoss: 4.2008e-01   valLoss:5.7316e+00  time: 1.40e+00
epoch: 5   trainLoss: 3.0435e-01   valLoss:1.0938e+01  time: 1.40e+00
epoch: 6   trainLoss: 2.5987e-01   valLoss:8.0616e+00  time: 1.42e+00
epoch: 7   trainLoss: 2.1944e-01   valLoss:3.1586e+00  time: 1.42e+00
epoch: 8   trainLoss: 1.6754e-01   valLoss:1.0471e+00  time: 1.41e+00
epoch: 9   trainLoss: 1.5883e-01   valLoss:7.5050e-01  time: 1.42e+00
epoch: 10   trainLoss: 1.1384e-01   valLoss:5.0900e-01  time: 1.40e+00
epoch: 11   trainLoss: 1.2255e-01   valLoss:7.2924e-01  time: 1.39e+00
epoch: 12   trainLoss: 1.0340e-01   valLoss:5.7143e-01  time: 1.41e+00
epoch: 13   trainLoss: 1.1264e-01   valLoss:5

alt.Chart(...)

testing on design_6

training on design_7
epoch: 0   trainLoss: 9.2624e-01   valLoss:9.4495e-01  time: 1.43e+00
epoch: 1   trainLoss: 6.9986e-01   valLoss:9.4951e-01  time: 1.42e+00
epoch: 2   trainLoss: 5.6555e-01   valLoss:9.7398e-01  time: 1.42e+00
epoch: 3   trainLoss: 4.7251e-01   valLoss:1.0158e+00  time: 1.42e+00
epoch: 4   trainLoss: 3.8179e-01   valLoss:1.0776e+00  time: 1.43e+00
epoch: 5   trainLoss: 3.3790e-01   valLoss:1.3269e+00  time: 1.44e+00
epoch: 6   trainLoss: 3.1862e-01   valLoss:1.4859e+00  time: 1.47e+00
epoch: 7   trainLoss: 2.8424e-01   valLoss:1.7119e+00  time: 1.43e+00
epoch: 8   trainLoss: 2.5288e-01   valLoss:5.3172e-01  time: 1.49e+00
epoch: 9   trainLoss: 2.1572e-01   valLoss:4.2538e-01  time: 1.46e+00
epoch: 10   trainLoss: 2.1278e-01   valLoss:4.8854e-01  time: 1.46e+00
epoch: 11   trainLoss: 1.8304e-01   valLoss:5.4178e-01  time: 1.46e+00
epoch: 12   trainLoss: 1.7193e-01   valLoss:6.8131e-01  time: 1.47e+00
epoch: 13   trainLoss: 1.9548e-01   valLoss:5

...

alt.Chart(...)

testing on design_8

training on design_9
epoch: 0   trainLoss: 1.0985e+00   valLoss:1.3870e+00  time: 1.61e+00
epoch: 1   trainLoss: 7.4587e-01   valLoss:1.2481e+00  time: 1.62e+00
epoch: 2   trainLoss: 5.0610e-01   valLoss:1.0988e+00  time: 1.62e+00
epoch: 3   trainLoss: 3.6199e-01   valLoss:1.1612e+00  time: 1.61e+00
epoch: 4   trainLoss: 3.0372e-01   valLoss:1.1361e+00  time: 1.65e+00
epoch: 5   trainLoss: 2.5260e-01   valLoss:8.1446e-01  time: 1.61e+00
epoch: 6   trainLoss: 2.2273e-01   valLoss:6.2702e-01  time: 1.62e+00
epoch: 7   trainLoss: 2.1476e-01   valLoss:3.7106e-01  time: 1.64e+00
epoch: 8   trainLoss: 2.1784e-01   valLoss:2.7541e-01  time: 1.64e+00
epoch: 9   trainLoss: 1.7872e-01   valLoss:2.6906e-01  time: 1.61e+00
epoch: 10   trainLoss: 1.5377e-01   valLoss:3.8349e-01  time: 1.61e+00
epoch: 11   trainLoss: 1.3602e-01   valLoss:3.9940e-01  time: 1.59e+00
epoch: 12   trainLoss: 1.3569e-01   valLoss:3.8571e-01  time: 1.63e+00
epoch: 13   trainLoss: 1.2828e-01   valLoss:3

alt.Chart(...)

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000002,0.001153,0.079541,0.002823,0.070391,0.172322,0.001489,test group,design_5
1,0.000011,0.002179,0.115890,0.010452,0.065323,0.313352,0.082382,test group,design_5
2,0.000012,0.002344,0.078194,0.009689,0.041208,0.170325,0.170325,test group,design_5
3,0.000011,0.002757,0.064746,0.007145,0.028447,0.073712,0.032357,test group,design_5
4,0.000008,0.002237,0.148454,0.007131,0.157606,0.502532,0.067912,test group,design_5
...,...,...,...,...,...,...,...,...,...
895,0.000018,0.002978,0.111516,0.016172,0.080251,0.435730,0.379596,test group,design_9
896,0.000048,0.005241,0.080118,0.016733,0.032673,0.104326,0.078019,test group,design_9
897,0.000027,0.004099,0.163260,0.010472,0.124005,0.316799,0.108192,test group,design_9
898,0.000018,0.003293,0.096201,0.010582,0.059856,0.192317,0.125219,test group,design_9


## 3. Train on all groups at once

In [5]:
allTrainData, allValData = [], []
print('training on all groups')
for name, data in trainSets.items():
    allTrainData = allTrainData + data
    allValData = allValData + valSets[name]

gcn = FeaStNet()
history = gcn.trainModel(allTrainData, allValData, epochs=epochs, saveDir=saveDir+'allGroups')

display(plotHistory(history))

# test
for testName, testSet in testSets.items():
    print('testing on '+testName+'\n')
    resultsDict = gcn.testModel(testSet, level='field')
    resultsDict['Trained on'] = ['all groups']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [testName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
    
pd.DataFrame(resultsList)

training on all groups
epoch: 0   trainLoss: 6.8000e-01   valLoss:1.2158e+00  time: 7.33e+00
epoch: 1   trainLoss: 3.5738e-01   valLoss:8.3354e-01  time: 7.30e+00
epoch: 2   trainLoss: 2.5532e-01   valLoss:6.8586e-01  time: 7.29e+00
epoch: 3   trainLoss: 1.9580e-01   valLoss:6.5061e-01  time: 7.36e+00
epoch: 4   trainLoss: 1.7614e-01   valLoss:5.4106e-01  time: 7.30e+00
epoch: 5   trainLoss: 1.4567e-01   valLoss:5.2198e-01  time: 7.35e+00
epoch: 6   trainLoss: 1.3839e-01   valLoss:3.5189e-01  time: 7.30e+00
epoch: 7   trainLoss: 1.1888e-01   valLoss:2.5415e-01  time: 7.34e+00
epoch: 8   trainLoss: 1.0960e-01   valLoss:4.4107e-01  time: 7.25e+00
epoch: 9   trainLoss: 1.0967e-01   valLoss:3.1399e-01  time: 7.18e+00
epoch: 10   trainLoss: 9.3120e-02   valLoss:2.0561e-01  time: 7.26e+00
epoch: 11   trainLoss: 8.4851e-02   valLoss:2.3624e-01  time: 7.28e+00
epoch: 12   trainLoss: 7.5381e-02   valLoss:2.6121e-01  time: 7.27e+00
epoch: 13   trainLoss: 8.4736e-02   valLoss:1.7254e-01  time: 7.

alt.Chart(...)

testing on design_5

testing on design_6

testing on design_7

testing on design_8

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000002,0.001153,0.079541,0.002823,0.070391,0.172322,0.001489,test group,design_5
1,0.000011,0.002179,0.115890,0.010452,0.065323,0.313352,0.082382,test group,design_5
2,0.000012,0.002344,0.078194,0.009689,0.041208,0.170325,0.170325,test group,design_5
3,0.000011,0.002757,0.064746,0.007145,0.028447,0.073712,0.032357,test group,design_5
4,0.000008,0.002237,0.148454,0.007131,0.157606,0.502532,0.067912,test group,design_5
...,...,...,...,...,...,...,...,...,...
1795,0.000002,0.001060,0.039179,0.004335,0.028554,0.116797,0.106816,all groups,design_9
1796,0.000050,0.006002,0.090927,0.013575,0.037419,0.084634,0.042521,all groups,design_9
1797,0.000012,0.002705,0.104957,0.010066,0.081848,0.304542,0.113062,all groups,design_9
1798,0.000016,0.003056,0.077679,0.008437,0.055537,0.153335,0.116053,all groups,design_9


## 4. Leave one out

In [6]:
checkptFiles = {}
for trainName, trainSet in trainSets.items():
    allTrainData, allValData = [], []
    print('training on all but '+ trainName)
    for name, data in trainSets.items():
        if name != trainName:
            allTrainData = allTrainData + data
            allValData = allValData + valSets[name]

    gcn = FeaStNet()
    history = gcn.trainModel(allTrainData, allValData, epochs=epochs, saveDir=saveDir+'allBut_'+ trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['all groups but test group']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
    
    # note checkpoint files for use in transfer learning
    checkptFiles[trainName] = gcn.checkptFile
    
pd.DataFrame(resultsList)

training on all but design_5
epoch: 0   trainLoss: 7.7299e-01   valLoss:9.9748e-01  time: 6.03e+00
epoch: 1   trainLoss: 4.1002e-01   valLoss:1.8921e+00  time: 5.89e+00
epoch: 2   trainLoss: 3.0343e-01   valLoss:4.5298e-01  time: 5.97e+00
epoch: 3   trainLoss: 2.5774e-01   valLoss:3.8528e-01  time: 5.89e+00
epoch: 4   trainLoss: 2.0453e-01   valLoss:3.5806e-01  time: 5.99e+00
epoch: 5   trainLoss: 1.7136e-01   valLoss:4.4907e-01  time: 5.88e+00
epoch: 6   trainLoss: 1.5412e-01   valLoss:4.5751e-01  time: 5.93e+00
epoch: 7   trainLoss: 1.3954e-01   valLoss:3.8658e-01  time: 5.90e+00
epoch: 8   trainLoss: 1.3525e-01   valLoss:5.1862e-01  time: 5.88e+00
epoch: 9   trainLoss: 1.1988e-01   valLoss:3.8575e-01  time: 5.83e+00
epoch: 10   trainLoss: 1.2736e-01   valLoss:3.8420e-01  time: 5.85e+00
epoch: 11   trainLoss: 1.1655e-01   valLoss:3.2736e-01  time: 5.91e+00
epoch: 12   trainLoss: 1.0682e-01   valLoss:2.8336e-01  time: 5.93e+00
epoch: 13   trainLoss: 9.2950e-02   valLoss:2.4150e-01  ti

alt.Chart(...)

testing on design_5

training on all but design_6
epoch: 0   trainLoss: 5.9125e-01   valLoss:1.3214e+00  time: 5.90e+00
epoch: 1   trainLoss: 3.3535e-01   valLoss:1.4752e+00  time: 5.90e+00
epoch: 2   trainLoss: 2.5225e-01   valLoss:4.1489e-01  time: 5.84e+00
epoch: 3   trainLoss: 2.2859e-01   valLoss:3.3881e-01  time: 5.90e+00
epoch: 4   trainLoss: 1.7298e-01   valLoss:3.5696e-01  time: 5.80e+00
epoch: 5   trainLoss: 1.5453e-01   valLoss:2.8971e-01  time: 5.83e+00
epoch: 6   trainLoss: 1.3768e-01   valLoss:3.6138e-01  time: 5.80e+00
epoch: 7   trainLoss: 1.2000e-01   valLoss:2.3868e-01  time: 5.78e+00
epoch: 8   trainLoss: 1.1174e-01   valLoss:2.4944e-01  time: 5.91e+00
epoch: 9   trainLoss: 1.0632e-01   valLoss:3.2773e-01  time: 5.85e+00
epoch: 10   trainLoss: 1.0409e-01   valLoss:1.8659e-01  time: 5.83e+00
epoch: 11   trainLoss: 9.4915e-02   valLoss:2.2021e-01  time: 5.81e+00
epoch: 12   trainLoss: 8.0354e-02   valLoss:1.8232e-01  time: 5.78e+00
epoch: 13   trainLoss: 8.8295e-02   v

alt.Chart(...)

testing on design_6

training on all but design_7
epoch: 0   trainLoss: 6.9766e-01   valLoss:1.1157e+00  time: 5.76e+00
epoch: 1   trainLoss: 3.8837e-01   valLoss:1.5075e+00  time: 5.79e+00
epoch: 2   trainLoss: 2.9510e-01   valLoss:4.0748e-01  time: 5.73e+00
epoch: 3   trainLoss: 2.2127e-01   valLoss:2.5479e-01  time: 5.78e+00
epoch: 4   trainLoss: 1.7581e-01   valLoss:2.2779e-01  time: 5.80e+00
epoch: 5   trainLoss: 1.3754e-01   valLoss:2.0443e-01  time: 5.72e+00
epoch: 6   trainLoss: 1.2336e-01   valLoss:1.6268e-01  time: 5.75e+00
epoch: 7   trainLoss: 1.1629e-01   valLoss:1.6653e-01  time: 5.71e+00
epoch: 8   trainLoss: 1.0075e-01   valLoss:1.6393e-01  time: 5.78e+00
epoch: 9   trainLoss: 9.7467e-02   valLoss:1.6590e-01  time: 5.82e+00
epoch: 10   trainLoss: 1.0700e-01   valLoss:2.0267e-01  time: 5.75e+00
epoch: 11   trainLoss: 1.0189e-01   valLoss:1.3607e-01  time: 5.78e+00
epoch: 12   trainLoss: 8.4004e-02   valLoss:1.4607e-01  time: 5.82e+00
epoch: 13   trainLoss: 8.8472e-02   v

...

alt.Chart(...)

testing on design_8

training on all but design_9
epoch: 0   trainLoss: 7.5363e-01   valLoss:1.0352e+00  time: 5.51e+00
epoch: 1   trainLoss: 4.2047e-01   valLoss:2.5468e+00  time: 5.51e+00
epoch: 2   trainLoss: 2.7567e-01   valLoss:5.6439e-01  time: 5.46e+00
epoch: 3   trainLoss: 2.1174e-01   valLoss:5.2091e-01  time: 5.42e+00
epoch: 4   trainLoss: 1.7258e-01   valLoss:5.6654e-01  time: 5.57e+00
epoch: 5   trainLoss: 1.4923e-01   valLoss:3.4970e-01  time: 5.50e+00
epoch: 6   trainLoss: 1.3589e-01   valLoss:3.9581e-01  time: 5.56e+00
epoch: 7   trainLoss: 1.5327e-01   valLoss:4.0095e-01  time: 5.58e+00
epoch: 8   trainLoss: 1.0481e-01   valLoss:3.5697e-01  time: 5.57e+00
epoch: 9   trainLoss: 1.0217e-01   valLoss:2.2735e-01  time: 5.56e+00
epoch: 10   trainLoss: 1.1063e-01   valLoss:2.9786e-01  time: 5.58e+00
epoch: 11   trainLoss: 1.2647e-01   valLoss:3.9159e-01  time: 5.60e+00
epoch: 12   trainLoss: 1.0533e-01   valLoss:3.2509e-01  time: 5.61e+00
epoch: 13   trainLoss: 8.1970e-02   v

alt.Chart(...)

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000002,0.001153,0.079541,0.002823,0.070391,0.172322,0.001489,test group,design_5
1,0.000011,0.002179,0.115890,0.010452,0.065323,0.313352,0.082382,test group,design_5
2,0.000012,0.002344,0.078194,0.009689,0.041208,0.170325,0.170325,test group,design_5
3,0.000011,0.002757,0.064746,0.007145,0.028447,0.073712,0.032357,test group,design_5
4,0.000008,0.002237,0.148454,0.007131,0.157606,0.502532,0.067912,test group,design_5
...,...,...,...,...,...,...,...,...,...
2695,0.000037,0.004215,0.132237,0.017692,0.113562,0.476707,0.114675,all groups but test group,design_9
2696,0.000201,0.010623,0.148666,0.032592,0.066228,0.203198,0.140929,all groups but test group,design_9
2697,0.000020,0.003544,0.132627,0.009920,0.107231,0.300115,0.256289,all groups but test group,design_9
2698,0.000120,0.007902,0.172779,0.027980,0.143605,0.508509,0.086468,all groups but test group,design_9


## 5. Transfer learning

In [7]:
allIds = list(range(len(trainSet)))
_, trainIds = train_test_split(allIds, test_size=0.2, shuffle=True, random_state=1234) # train on only 25%

for trainName, trainSet in trainSets.items():
    # load pre-trained model and train on new data
    print('transfer learning on '+trainName)
    gcn = FeaStNet()
    smallTrainSet = [trainSet[i] for i in trainIds]
    history = gcn.trainModel(smallTrainSet, valSets[trainName], restartFile=checkptFiles[trainName], 
                             epochs=epochs, saveDir=saveDir+'transferLearn_'+ trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['transfer learning (20%)']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
        
pd.DataFrame(resultsList)

transfer learning on design_5
loading restart file
epoch: 0   trainLoss: 1.0744e+00   valLoss:2.3175e+00  time: 8.66e-01
epoch: 1   trainLoss: 5.7587e-01   valLoss:3.2366e+00  time: 8.83e-01
epoch: 2   trainLoss: 3.9318e-01   valLoss:3.0304e+00  time: 8.49e-01
epoch: 3   trainLoss: 2.6249e-01   valLoss:3.1366e+00  time: 8.72e-01
epoch: 4   trainLoss: 1.8142e-01   valLoss:1.7711e+00  time: 8.73e-01
epoch: 5   trainLoss: 1.1496e-01   valLoss:1.1681e+00  time: 8.53e-01
epoch: 6   trainLoss: 9.6440e-02   valLoss:1.0423e+00  time: 8.49e-01
epoch: 7   trainLoss: 7.3605e-02   valLoss:7.1957e-01  time: 8.58e-01
epoch: 8   trainLoss: 6.9805e-02   valLoss:4.2427e-01  time: 8.69e-01
epoch: 9   trainLoss: 6.4705e-02   valLoss:3.3242e-01  time: 8.59e-01
epoch: 10   trainLoss: 5.7546e-02   valLoss:2.7822e-01  time: 8.64e-01
epoch: 11   trainLoss: 5.5795e-02   valLoss:2.0710e-01  time: 8.73e-01
epoch: 12   trainLoss: 4.9704e-02   valLoss:1.6885e-01  time: 8.75e-01
epoch: 13   trainLoss: 4.1202e-02   

alt.Chart(...)

testing on design_5

transfer learning on design_6
loading restart file
epoch: 0   trainLoss: 7.3222e-01   valLoss:7.2236e-01  time: 9.00e-01
epoch: 1   trainLoss: 3.7222e-01   valLoss:1.4702e+00  time: 8.89e-01
epoch: 2   trainLoss: 2.0197e-01   valLoss:1.4551e+00  time: 8.81e-01
epoch: 3   trainLoss: 1.2634e-01   valLoss:1.2197e+00  time: 8.61e-01
epoch: 4   trainLoss: 9.8479e-02   valLoss:1.0178e+00  time: 8.49e-01
epoch: 5   trainLoss: 8.2552e-02   valLoss:9.0587e-01  time: 8.56e-01
epoch: 6   trainLoss: 6.0502e-02   valLoss:7.0623e-01  time: 8.55e-01
epoch: 7   trainLoss: 4.4438e-02   valLoss:4.8603e-01  time: 8.63e-01
epoch: 8   trainLoss: 3.9570e-02   valLoss:4.1888e-01  time: 8.53e-01
epoch: 9   trainLoss: 4.4061e-02   valLoss:3.6438e-01  time: 8.53e-01
epoch: 10   trainLoss: 4.0564e-02   valLoss:1.8007e-01  time: 8.86e-01
epoch: 11   trainLoss: 3.3288e-02   valLoss:1.1146e-01  time: 8.83e-01
epoch: 12   trainLoss: 2.8352e-02   valLoss:8.7667e-02  time: 8.76e-01
epoch: 13   tra

alt.Chart(...)

testing on design_6

transfer learning on design_7
loading restart file
epoch: 0   trainLoss: 1.7231e-01   valLoss:4.2390e-01  time: 8.90e-01
epoch: 1   trainLoss: 3.2263e-01   valLoss:1.0573e+00  time: 8.96e-01
epoch: 2   trainLoss: 1.3404e-01   valLoss:1.1733e+00  time: 8.95e-01
epoch: 3   trainLoss: 1.1048e-01   valLoss:1.2836e+00  time: 8.92e-01
epoch: 4   trainLoss: 8.6561e-02   valLoss:6.5792e-01  time: 8.94e-01
epoch: 5   trainLoss: 7.5715e-02   valLoss:3.9733e-01  time: 8.99e-01
epoch: 6   trainLoss: 6.4987e-02   valLoss:2.6148e-01  time: 8.81e-01
epoch: 7   trainLoss: 5.7915e-02   valLoss:2.7004e-01  time: 9.13e-01
epoch: 8   trainLoss: 5.0611e-02   valLoss:3.0173e-01  time: 8.93e-01
epoch: 9   trainLoss: 4.3505e-02   valLoss:2.2773e-01  time: 8.86e-01
epoch: 10   trainLoss: 3.9515e-02   valLoss:1.7881e-01  time: 9.08e-01
epoch: 11   trainLoss: 3.4076e-02   valLoss:1.5155e-01  time: 8.91e-01
epoch: 12   trainLoss: 2.9668e-02   valLoss:1.1749e-01  time: 8.91e-01
epoch: 13   tra

...

alt.Chart(...)

testing on design_8

transfer learning on design_9
loading restart file
epoch: 0   trainLoss: 1.5828e+00   valLoss:3.4858e+00  time: 9.19e-01
epoch: 1   trainLoss: 1.3396e+00   valLoss:4.3063e+00  time: 9.40e-01
epoch: 2   trainLoss: 8.9345e-01   valLoss:4.6652e+00  time: 9.27e-01
epoch: 3   trainLoss: 7.2993e-01   valLoss:3.8879e+00  time: 9.36e-01
epoch: 4   trainLoss: 5.9337e-01   valLoss:3.1694e+00  time: 9.37e-01
epoch: 5   trainLoss: 5.0626e-01   valLoss:2.7646e+00  time: 9.26e-01
epoch: 6   trainLoss: 4.7104e-01   valLoss:2.7230e+00  time: 9.33e-01
epoch: 7   trainLoss: 4.5843e-01   valLoss:1.5557e+00  time: 9.22e-01
epoch: 8   trainLoss: 3.3646e-01   valLoss:1.7636e+00  time: 9.50e-01
epoch: 9   trainLoss: 3.4095e-01   valLoss:1.5759e+00  time: 9.54e-01
epoch: 10   trainLoss: 2.7354e-01   valLoss:1.3915e+00  time: 9.26e-01
epoch: 11   trainLoss: 2.4956e-01   valLoss:1.0720e+00  time: 9.35e-01
epoch: 12   trainLoss: 2.3279e-01   valLoss:6.7573e-01  time: 9.59e-01
epoch: 13   tra

alt.Chart(...)

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000002,0.001153,0.079541,0.002823,0.070391,0.172322,0.001489,test group,design_5
1,0.000011,0.002179,0.115890,0.010452,0.065323,0.313352,0.082382,test group,design_5
2,0.000012,0.002344,0.078194,0.009689,0.041208,0.170325,0.170325,test group,design_5
3,0.000011,0.002757,0.064746,0.007145,0.028447,0.073712,0.032357,test group,design_5
4,0.000008,0.002237,0.148454,0.007131,0.157606,0.502532,0.067912,test group,design_5
...,...,...,...,...,...,...,...,...,...
3595,0.000020,0.003448,0.127520,0.010897,0.092898,0.293619,0.293619,transfer learning (20%),design_9
3596,0.000057,0.005299,0.065505,0.017137,0.033038,0.106841,0.026611,transfer learning (20%),design_9
3597,0.000043,0.004596,0.187285,0.020325,0.139054,0.614911,0.016224,transfer learning (20%),design_9
3598,0.000014,0.002710,0.070793,0.010307,0.049257,0.187315,0.122171,transfer learning (20%),design_9


In [8]:
# save test results to file
df = pd.DataFrame(resultsList)
df.to_csv(saveDir+'testResults.csv', index=False)

In [9]:
# load test results from file
df = pd.read_csv(saveDir+'testResults.csv')
df

,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000002,0.001153,0.079541,0.002823,0.070391,0.172322,0.001489,test group,design_5
1,0.000011,0.002179,0.115890,0.010452,0.065323,0.313352,0.082382,test group,design_5
2,0.000012,0.002344,0.078194,0.009689,0.041208,0.170325,0.170325,test group,design_5
3,0.000011,0.002757,0.064746,0.007145,0.028447,0.073712,0.032357,test group,design_5
4,0.000008,0.002237,0.148454,0.007131,0.157606,0.502532,0.067912,test group,design_5
...,...,...,...,...,...,...,...,...,...
3595,0.000020,0.003448,0.127520,0.010897,0.092898,0.293619,0.293619,transfer learning (20%),design_9
3596,0.000057,0.005299,0.065505,0.017137,0.033038,0.106841,0.026611,transfer learning (20%),design_9
3597,0.000043,0.004596,0.187285,0.020325,0.139054,0.614911,0.016224,transfer learning (20%),design_9
3598,0.000014,0.002710,0.070793,0.010307,0.049257,0.187315,0.122171,transfer learning (20%),design_9


## 6. Plot results

In [10]:
order = ['test group', 'all groups']
barChart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Trained on:N', sort=order, title='', axis=alt.Axis(ticks=False, labels=False)),
    y=alt.Y('mean(mse):Q', scale=alt.Scale(type='log'), axis=alt.Axis(tickCount=8, format=".0e"), title='MSE'),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.8),
    tooltip='mean(mse):Q'
).properties(width=75, height=200)

scatter = alt.Chart(df).mark_circle(size=20).encode(
    x=alt.X('Trained on:N', title='', sort=order),
    y=alt.Y('mse:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.3),
    tooltip='mse:Q'
)

alt.layer(barChart, scatter, data=df).facet(
    column=alt.Column('Tested on:N'))

alt.FacetChart(...)

In [11]:
order = ['test group', 'all groups']
barChart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Trained on:N', sort=order, title='', axis=alt.Axis(ticks=False, labels=False)),
    y=alt.Y('mean(mse):Q', axis=alt.Axis(tickCount=8, format=".0e"), title='MSE'),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.8),
    tooltip='mean(mse):Q'
).properties(width=75, height=200)

scatter = alt.Chart(df).mark_circle(size=20).encode(
    x=alt.X('Trained on:N', title='', sort=order),
    y=alt.Y('mse:Q'),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.3),
    tooltip='mse:Q'
)

alt.layer(barChart, scatter, data=df).facet(
    column=alt.Column('Tested on:N'))

alt.FacetChart(...)

In [13]:
order = ['test group', 'all groups']
barChart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Trained on:N', sort=order, title='', axis=alt.Axis(ticks=False, labels=False)),
    y=alt.Y('mean(mse):Q', axis=alt.Axis(tickCount=8, format=".0e"), title='MSE'),
    color=alt.Color('Trained on:N', sort=order, legend=alt.Legend(orient='bottom')),
    opacity = alt.OpacityValue(0.8),
    tooltip='mean(mse):Q'
).properties(width=75, height=200)

alt.layer(barChart, data=df).facet(
    column=alt.Column('Tested on:N'))

alt.FacetChart(...)